In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b


In [4]:
from langgraph.prebuilt import ToolNode

tool_list = [add, multiply]
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)


In [ ]:
from langgraph.graph import MessagesState, StateGraph

# MessageState
# class MessagesState(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]
# add_messages로 HumanMessage, AIMessage, ToolMessage를 자동으로 추가

graph_builder = StateGraph(MessagesState)

In [ ]:
def agent(state: MessagesState) -> MessagesState:
    messages = state['messages']
    
    # AIMessage
    response = llm_with_tools.invoke(messages)
    
    # add_messages로 자동으로 append
    return {'messages': [response]}

In [ ]:
from typing import Literal
from langgraph.graph import END

def should_continue(state: MessagesState) -> Literal['tools', END]:
    # 답변이 만들어졌으면 content안에 답변이 들어가고, tool_calls 속성이 존재하지 않는다.
    # 답변이 안만들어졌으면 content안에 답변이 없고, tool_calls 속성에 사용해 볼 tool이 들어간다. 
    messages = state['messages']
    
    last_ai_message = messages[-1]
    
    if last_ai_message.tool_calls:
        return 'tools'
    
    return END

In [ ]:
from langgraph.graph import START, END

graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)


graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges(
    'agent',
    should_continue,
    ['tools', END] # 가능한 분기 후보 목록을 dict로 매핑시키는게 아니라 그냥 이렇게 리스트로 나열해도 된다. 
)
graph_builder.add_edge('tools', 'agent')

In [ ]:
graph= graph_builder.compile()

In [1]:
from langchain_core.messages import HumanMessage

query = '3에다 5를 더하고 거기에 8을 곱하면?'
initial_state = {'messages': [HumanMessage(query)]}

for chunk in graph.stream(initial_state, stream_mode='values'):
    chunk['messages'][-1].pretty_print()

NameError: name 'graph' is not defined